<a href="https://colab.research.google.com/github/karandomguy/HCR/blob/main/HCR(test).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import cv2
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import regularizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Activation
from google.colab import files

In [2]:
! pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
!cp '/content/drive/MyDrive/HCR/kaggle.json' '/content'

In [10]:
import zipfile
import os

os.environ['KAGGLE_CONFIG_DIR'] = "/content"

In [11]:
!kaggle datasets download -d sankalpsrivastava26/capital-alphabets-28x28

100% 3.17G/3.17G [00:28<00:00, 144MB/s]
100% 3.17G/3.17G [00:28<00:00, 120MB/s]


In [12]:
zip_ref = zipfile.ZipFile('capital-alphabets-28x28.zip', 'r') #Opens the zip file in read mode
zip_ref.extractall('/tmp') #Extracts the files into the /tmp folder
zip_ref.close()

In [13]:
len(os.listdir('/tmp/datasetDigit/train/'))

10

In [14]:
x_train=os.listdir('/tmp/dataset/train/')

In [15]:
print(x_train)

['S', 'R', 'F', 'J', 'G', 'D', 'O', 'H', 'K', 'M', 'N', 'U', 'V', 'L', 'Q', 'Y', 'W', 'I', 'T', 'Z', 'E', 'P', 'A', 'C', 'B', 'X']


In [16]:
!ls /tmp/dataset/train

A  B  C  D  E  F  G  H	I  J  K  L  M  N  O  P	Q  R  S  T  U  V  W  X	Y  Z


In [17]:
print(x_train)

['S', 'R', 'F', 'J', 'G', 'D', 'O', 'H', 'K', 'M', 'N', 'U', 'V', 'L', 'Q', 'Y', 'W', 'I', 'T', 'Z', 'E', 'P', 'A', 'C', 'B', 'X']


In [18]:
train_datagen = ImageDataGenerator(rescale=1./255)
train = train_datagen.flow_from_directory(
    directory = "/tmp/dataset/train",
    target_size=(300, 300))

Found 150341 images belonging to 26 classes.


In [19]:
val = train_datagen.flow_from_directory(
    directory = "/tmp/dataset/validation",
    target_size=(300, 300))

Found 43250 images belonging to 26 classes.


In [20]:
test = train_datagen.flow_from_directory(
    directory = "/tmp/dataset/test",
    target_size=(300, 300))

Found 43121 images belonging to 26 classes.


In [21]:
model = tf.keras.models.Sequential([
    #input shape is the desired size of the image 300x300 with 3 bytes color
    # First convolution
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(300, 300, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # Second convolution
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Third convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Fourth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Fifth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    # 26 output neuron.
    tf.keras.layers.Dense(26, activation='softmax')
])
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


In [22]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 298, 298, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 149, 149, 16)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 147, 147, 32)      4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 73, 73, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 71, 71, 64)        18496     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 35, 35, 64)       0

In [23]:
X_val,y_val = next(val)

In [24]:
print(X_val.shape,y_val.shape)

(32, 300, 300, 3) (32, 26)


In [25]:
checkpointer = EarlyStopping(monitor = 'val_accuracy', verbose = 1, restore_best_weights=True,mode="max",patience = 25)

In [27]:
history = model.fit(
            train ,
            steps_per_epoch =len(train)//32, #batch size is 32
            epochs=10,
            verbose=1,
            validation_data=(X_val,y_val),
            callbacks = [checkpointer])

Epoch 1/10
  2/146 [..............................] - ETA: 11:37 - loss: 2.8913 - accuracy: 0.2344

KeyboardInterrupt: ignored

In [ ]:
training_loss = history.history['loss']
val_loss = history.history['val_loss']

# Create count of the number of epochs
epoch_count = range(1, len(training_loss) + 1)

# Visualize loss history
plt.plot(epoch_count, training_loss, 'r--')
plt.plot(epoch_count, val_loss, 'b-')
plt.legend(['Training Loss', 'Val Loss'])
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show()

In [ ]:
training_accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']

# Create count of the number of epochs
epoch_count = range(1, len(training_accuracy) + 1)

# Visualize loss history

plt.plot(epoch_count, training_accuracy, 'r--')
plt.plot(epoch_count, val_accuracy, 'b-')
plt.legend(['Training Accuracy', 'Val Accuracy'])
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.show()